# Stima dei livelli di obesità in base alle abitudini alimentari e alle condizioni fisiche
Progetto di Programmazione di Applicazioni Data Intensive - 2022/2023

**Realizzato da:** Fabio Veroli fabio.veroli@studio.unibo.it

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Caricamento dei dati
L'obbiettivo del problema che andremo a studiare è stimare il livello di obesità di una persona in base alle sue abitudini alimentari e alle sue condizioni fisiche. 

Il dataset utilizzato, consultabile presso [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition), riguarda i dati di circa 2000 individui provenienti da Messico, Peru e Colombia; il 23% dei dati è stato ottenuto da un sondaggio online, mentre il restante 77% è stato generato sinteticamente utilizzando lo strumento Weka e il filtro SMOTE. 

In [122]:
df = pd.read_csv("https://raw.githubusercontent.com/Fab-Ver/DataIntensive-Project/main/data/ObesityDataSet.csv",delimiter=',')
df.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


### Significato delle colonne
Le feature presenti nel dataset sono: 
- `Gender`: genere dell'individuo. Possibili valori: Female o Male
- `Age`: età dell'individuo (valore numerico)
- `Height`: altezza dell'individuo in metri (valore numerico)
- `Weight`: peso dell'individuo in kilogrammi (valore numerico)
- `family_history_with_overweight`: indica se nella famiglia dell'individuo sono presenti altri familiari che soffrono di obesità. Possibili valori: yes o no
- `FAVC`: indica se l'individuo consuma frequentemente cibi ad alto contenuto calorico. Possibili valori: yes o no
- `FCVC`: indica quanto spesso l'individuo mangia verdure durante i pasti. Possibili valori: Never (1), Sometimes (2), Always (3)
- `NCP`: numero di pasti principali consumati dall'individuo in un giorno (valore numerico)
- `CAEC`: indica quanto spesso l'individuo consuma cibi tra due pasti. Possibili valori: No, Sometimes, Frequently, Always
- `SMOKE`: indica se l'individuo fuma. Possibili valori: yes o no
- `CH20`: indica il consumo giornaliero di acqua. Possibili valori: less than a liter (1), between 1 and 2L (2), more than 2L (3)
- `SCC`: indica se l'individuo monitora le calorie che assume giornalemente. Possibili valori: yes o no
- `FAF`: indica il numero di giorni in una settimana in cui l'individuo effettua attività fisica. Possibili valori: never (0), 1-2 days (1), 2-4 days (2), 4-5 days (3)
- `TUE`: indica il numero di ore giornaliere passate utilizzando dispositivi tecnologici come cellulare, videogiochi, televisione, computer. Possibili valori: 0-2 hours (0), 3-5 hours (1), more than 5 hours (2)
- `CALC`: indica quanto spesso l'individuo consuma alcol. Possibili valori: No, Sometimes, Frequently, Always
- `MTRANS`: tipo di mezzo di trasporto utilizzato dall'individuo. Possibili valori: 'Public_Transportation', 'Walking', 'Automobile', 'Motorbike', 'Bike'

La variabile che tenteremo di predirre è: 
- `NObeyesdad`: classificazione dei livelli di obesità basata sull'indice di massa corporea (BMI) secondo la WHO e la normativa messicana.

Modificihiamo i nomi delle colonne per renderli più esplicativi al fine dello studio del problema. 

In [123]:
col_names = {
    "Gender" : "gender",
    "Age" : "age",
    "Height" : "height",
    "Weight" : "weight",
    "family_history_with_overweight" : "overweight_in_family",
    "FAVC" : "high_caloric_food_frequently",
    "FCVC" : "freq_vegetables_cons",
    "NCP" : "num_meals",
    "CAEC" : "freq_cons_food_between_meals",
    "SMOKE" : "smoke",
    "CH2O" : "water_cons",
    "SCC" : "cal_cons_monitoring",
    "FAF" : "freq_physical_activity",
    "TUE" : "time_using_technology",
    "CALC" : "freq_alcohol_cons",
    "MTRANS" : "transportation",
    "NObeyesdad": "BMI_classification",
}
df.rename(columns=col_names,inplace=True)

In [124]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   gender                        2111 non-null   object 
 1   age                           2111 non-null   float64
 2   height                        2111 non-null   float64
 3   weight                        2111 non-null   float64
 4   overweight_in_family          2111 non-null   object 
 5   high_caloric_food_frequently  2111 non-null   object 
 6   freq_vegetables_cons          2111 non-null   float64
 7   num_meals                     2111 non-null   float64
 8   freq_cons_food_between_meals  2111 non-null   object 
 9   smoke                         2111 non-null   object 
 10  water_cons                    2111 non-null   float64
 11  cal_cons_monitoring           2111 non-null   object 
 12  freq_physical_activity        2111 non-null   float64
 13  tim

Dall'analisi dell'utilizzo della memoria emerge che il dataframe occupa molto spazio in memoria a causa di alcuni attributi rappresentati come oggetti. Passiamo quindi a classificare gli attributi in categorici, booleani e numerici, in modo da rappresentare gli attributi con tipi di dati più efficienti. 

In [125]:
cat_col = ["gender", "freq_vegetables_cons", "freq_cons_food_between_meals", 
           "water_cons", "freq_alcohol_cons", "freq_physical_activity", "time_using_technology",
           "transportation", "BMI_classification"]
bool_col = ["overweight_in_family", "high_caloric_food_frequently", "smoke", "cal_cons_monitoring"]
int_col = ["age", "num_meals"]

Convertiamo tutte gli attributi con valori possibili `yes` o `no` in attributi booleani.

In [126]:
mapping = {"yes" : True, "no" : False}
for col_name in bool_col: 
    df[col_name] = df[col_name].map(mapping)

Per le variabili `age` e `num_meals` tronchiamo i valori decimali in modo da ottenere un valore intero.

In [127]:
for col_name in int_col:
    df[col_name] = df[col_name].astype(int)

Alcuni attributi catagorici sono rappresentati in formato numerico, mentre altre in formato di stringa. Rappresentiamo tutti gli attributi che rappresentano una quantità misurabile utilizzando dei numeri crescenti. Per alcuni attributi già in formato numerico è necessario effettuare una conversione in quanto i dati importati risultano incorretti.  

In [128]:
mapping_cat_to_num = {"no" : 0, "Sometimes" : 1, "Frequently" : 2, "Always" : 3}
df["freq_cons_food_between_meals"] = df["freq_cons_food_between_meals"].map(mapping_cat_to_num)
df["freq_alcohol_cons"] = df["freq_alcohol_cons"].map(mapping_cat_to_num)

for col_name in ["freq_vegetables_cons","water_cons", "freq_physical_activity", "time_using_technology"]:
    df[col_name] = df[col_name].astype(int)

for col_name in cat_col:
    df[col_name] = df[col_name].astype('category')

In [129]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   gender                        2111 non-null   category
 1   age                           2111 non-null   int32   
 2   height                        2111 non-null   float64 
 3   weight                        2111 non-null   float64 
 4   overweight_in_family          2111 non-null   bool    
 5   high_caloric_food_frequently  2111 non-null   bool    
 6   freq_vegetables_cons          2111 non-null   category
 7   num_meals                     2111 non-null   int32   
 8   freq_cons_food_between_meals  2111 non-null   category
 9   smoke                         2111 non-null   bool    
 10  water_cons                    2111 non-null   category
 11  cal_cons_monitoring           2111 non-null   bool    
 12  freq_physical_activity        2111 non-null   ca

In [130]:
df.head()

,gender,age,height,weight,overweight_in_family,high_caloric_food_frequently,freq_vegetables_cons,num_meals,freq_cons_food_between_meals,smoke,water_cons,cal_cons_monitoring,freq_physical_activity,time_using_technology,freq_alcohol_cons,transportation,BMI_classification
0,Female,21,1.62,64.0,True,False,2,3,1,False,2,False,0,1,0,Public_Transportation,Normal_Weight
1,Female,21,1.52,56.0,True,False,3,3,1,True,3,True,3,0,1,Public_Transportation,Normal_Weight
2,Male,23,1.80,77.0,True,False,2,3,1,False,2,False,2,1,2,Public_Transportation,Normal_Weight
3,Male,27,1.80,87.0,False,False,3,3,1,False,2,False,2,0,2,Walking,Overweight_Level_I
4,Male,22,1.78,89.8,False,False,2,1,1,False,2,False,0,0,1,Public_Transportation,Overweight_Level_II


## Analisi Esplorativa

In [145]:
df.describe()

,age,height,weight,num_meals
count,2111.000000,2111.000000,2111.000000,2111.000000
mean,23.972525,1.701677,86.586058,2.523449
std,6.308664,0.093305,26.191172,0.830288
min,14.000000,1.450000,39.000000,1.000000
25%,19.000000,1.630000,65.473343,2.000000
50%,22.000000,1.700499,83.000000,3.000000
75%,26.000000,1.768464,107.430682,3.000000
max,61.000000,1.980000,173.000000,4.000000
